In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import time

In [2]:
gameRanks=pd.read_csv("GameRanks.csv")
gameIds= gameRanks['ID']

In [3]:
rootpath = 'https://boardgamegeek.com/xmlapi2/'
gamepath = 'thing/'
playpath = 'plays/'

In [37]:
gameparameters = {'id':gameIds[34],
              'stats' : 1}

playparameters = {"id": gameIds[34],
                  'type': 'thing',
                  'mindate':'2024-02-25'}

In [4]:
unrankedgames = gameRanks[gameRanks['Overall Rank'].isna()]['ID']

In [58]:
unrankedgames

100      7854
101    163412
102    287954
103    249259
104     31999
        ...  
746     15369
747    325348
748    303551
749    215565
750     67600
Name: ID, Length: 651, dtype: int64

In [23]:
gamePlays=[]
for id in gameIds[0:300]:
    playparameters = {"id": id, 
                      'type': 'thing',
                      'mindate':'2024-02-25'}
    r=requests.get(rootpath+playpath, params= playparameters)
    if r.status_code==429:
        time.sleep(30)
        r=requests.get(rootpath+playpath, params= playparameters)
        if r.status_code!=200: 
            print(f"Error Code:{r.status_code}, gameId {id}")
            break

    soup = BeautifulSoup(r.text, features='xml')
    gamePlays += re.findall('total="(\d+)"', str(soup.find('plays')))
    time.sleep(5)

In [27]:
#took a break, now resuming
for id in gameIds[300:399]:
    playparameters = {"id": id, 
                      'type': 'thing',
                      'mindate':'2024-02-25'}
    r=requests.get(rootpath+playpath, params= playparameters)
    if r.status_code==429:
        time.sleep(30)
        r=requests.get(rootpath+playpath, params= playparameters)
        if r.status_code!=200: 
            print(f"Error Code:{r.status_code}, gameId {id}")
            break

    soup = BeautifulSoup(r.text, features='xml')
    gamePlays += re.findall('total="(\d+)"', str(soup.find('plays')))
    time.sleep(5)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [29]:
#Lost connection
#Restarting
for id in gameIds[399:]:
    playparameters = {"id": id, 
                      'type': 'thing',
                      'mindate':'2024-02-25'}
    r=requests.get(rootpath+playpath, params= playparameters)
    if r.status_code==429:
        time.sleep(30)
        r=requests.get(rootpath+playpath, params= playparameters)
        if r.status_code!=200: 
            print(f"Error Code:{r.status_code}, gameId {id}")
            break

    soup = BeautifulSoup(r.text, features='xml')
    gamePlays += re.findall('total="(\d+)"', str(soup.find('plays')))
    time.sleep(5)

In [35]:
gameRanks['Recorded Plays Last Month']=gamePlays

gameRanks=gameRanks[['ID', 'Recorded Plays Last Month', 'Overall Rank', 'Abstract Rank',
                     'Customizable Rank', 'Thematic Rank', 'Family Rank', 'Children Rank',
                     'Party Rank', 'Strategy Rank', 'War Rank']]

In [37]:
#Update CSV so I don't lose everything
#gameRanks.to_csv('GameRanks.csv')

In [16]:
playparameters = {"id": gameIds[0:2], 
                  'type': 'thing',
                  'mindate':'2024-02-25'}

r= requests.get(rootpath+playpath, params= playparameters)    
r.status_code

200

In [17]:
soup = BeautifulSoup(r.text, features='xml')

In [22]:
soup.find('plays')

<plays page="1" termsofuse="https://boardgamegeek.com/xmlapi/termsofuse" total="1703">
<play date="2567-02-25" id="81885834" incomplete="0" length="0" location="" nowinstats="0" quantity="1" userid="3358389">
<item name="Brass: Birmingham" objectid="224517" objecttype="thing">
<subtypes>
<subtype value="boardgame"/>
<subtype value="boardgameimplementation"/>
</subtypes>
</item>
</play>
<play date="2567-02-24" id="81885843" incomplete="0" length="0" location="" nowinstats="0" quantity="1" userid="3358389">
<item name="Brass: Birmingham" objectid="224517" objecttype="thing">
<subtypes>
<subtype value="boardgame"/>
<subtype value="boardgameimplementation"/>
</subtypes>
</item>
</play>
<play date="2567-02-22" id="81753532" incomplete="0" length="0" location="" nowinstats="0" quantity="1" userid="3358389">
<item name="Brass: Birmingham" objectid="224517" objecttype="thing">
<subtypes>
<subtype value="boardgame"/>
<subtype value="boardgameimplementation"/>
</subtypes>
</item>
</play>
<play d

In [18]:
plays = soup.find_all('plays')
numplays=[]
for play in plays:
    numplays+=re.findall('total="(\d+)"', str(play))

In [19]:
numplays

['1703']

In [20]:
soup.prettify()

'<?xml version="1.0" encoding="utf-8"?>\n<plays page="1" termsofuse="https://boardgamegeek.com/xmlapi/termsofuse" total="1703">\n <play date="2567-02-25" id="81885834" incomplete="0" length="0" location="" nowinstats="0" quantity="1" userid="3358389">\n  <item name="Brass: Birmingham" objectid="224517" objecttype="thing">\n   <subtypes>\n    <subtype value="boardgame"/>\n    <subtype value="boardgameimplementation"/>\n   </subtypes>\n  </item>\n </play>\n <play date="2567-02-24" id="81885843" incomplete="0" length="0" location="" nowinstats="0" quantity="1" userid="3358389">\n  <item name="Brass: Birmingham" objectid="224517" objecttype="thing">\n   <subtypes>\n    <subtype value="boardgame"/>\n    <subtype value="boardgameimplementation"/>\n   </subtypes>\n  </item>\n </play>\n <play date="2567-02-22" id="81753532" incomplete="0" length="0" location="" nowinstats="0" quantity="1" userid="3358389">\n  <item name="Brass: Birmingham" objectid="224517" objecttype="thing">\n   <subtypes>\n

In [15]:
gameIds[0:2]

0    224517
1    161936
Name: ID, dtype: int64